In [1]:
from datetime import date
import pandas as pd
import nselib
from nselib import capital_market
from sqlalchemy import create_engine
import numpy as np
import os


In [2]:
import duckdb

motherduck_token = os.getenv("MOTHERDUCK_TOKEN")
con = duckdb.connect(f"md:?motherduck_token={motherduck_token}")

In [4]:
con.sql("SELECT date1,count(*) from daily_nse_price group by date1 order by date1 desc").df()

,date1,count_star()
0,2025-09-22,3039
1,2025-09-19,2979
2,2025-09-18,2984
3,2025-09-17,3007
4,2025-09-16,2979
...,...,...
1161,2021-01-07,2017
1162,2021-01-06,2040
1163,2021-01-05,2049
1164,2021-01-04,2055


In [3]:
allstocks= pd.read_excel("data/allstocks.xlsx")

In [40]:
NSE_ISIN = pd.read_csv("data/NSE_ISIN.csv")
NSE_ISIN.columns = NSE_ISIN.columns.str.strip()

In [4]:
df = con.sql("SELECT symbol as 'Ticker', date1 as 'Date', open_price as 'Open', high_price as 'High',low_price as 'Low',close_price as 'Close', deliv_qty as 'Volume' FROM daily_nse_price where series = 'EQ' and date1 > '2025-01-01'").df()

df["Volume"] = df["Volume"].astype(float)

In [5]:
def weekly_supertrend_daily(df, atr_period=10, multiplier=3):
    """
    df: DataFrame with ['Ticker','Date','Open','High','Low','Close']
    Returns df with 'SuperTrend' and 'Trend' columns applied to daily rows, based on weekly ATR
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Ticker','Date'])

    result_list = []

    for ticker, group in df.groupby('Ticker'):
        # Resample weekly to get High, Low, Close for SuperTrend calculation
        weekly = group.resample('W-FRI', on='Date').agg({
            'High':'max',
            'Low':'min',
            'Close':'last'
        }).sort_index()

        # Calculate weekly ATR
        weekly['H-L'] = weekly['High'] - weekly['Low']
        weekly['H-Cp'] = abs(weekly['High'] - weekly['Close'].shift())
        weekly['L-Cp'] = abs(weekly['Low'] - weekly['Close'].shift())
        weekly['TR'] = weekly[['H-L','H-Cp','L-Cp']].max(axis=1)
        weekly['ATR'] = weekly['TR'].rolling(atr_period, min_periods=1).mean()

        # Basic bands
        weekly['Basic_Up'] = (weekly['High'] + weekly['Low'])/2 + multiplier*weekly['ATR']
        weekly['Basic_Down'] = (weekly['High'] + weekly['Low'])/2 - multiplier*weekly['ATR']

        # Weekly SuperTrend
        weekly['SuperTrend'] = 0
        for i in range(1,len(weekly)):
            prev = weekly.iloc[i-1]
            if prev['SuperTrend'] < prev['Close']:
                curr_st = max(weekly.iloc[i]['Basic_Down'], prev['SuperTrend'])
            else:
                curr_st = min(weekly.iloc[i]['Basic_Up'], prev['SuperTrend'])
            weekly.iloc[i, weekly.columns.get_loc('SuperTrend')] = curr_st

        # ---- 2. Map weekly SuperTrend to daily data ----
        group = group.set_index('Date')
        group['SuperTrend'] = weekly['SuperTrend'].reindex(group.index, method='ffill')

        # ---- 3. Daily Trend based on daily Close vs weekly SuperTrend ----
        group['Trend'] = group['Close'] > group['SuperTrend']

        group['Ticker'] = ticker
        result_list.append(group.reset_index())

    return pd.concat(result_list, ignore_index=True)

In [6]:
daily_with_weekly_st = weekly_supertrend_daily(df, atr_period=10, multiplier=2)


/tmp/ipykernel_5244/4251973704.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '145.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weekly.iloc[i, weekly.columns.get_loc('SuperTrend')] = curr_st
/tmp/ipykernel_5244/4251973704.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '60.58000000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weekly.iloc[i, weekly.columns.get_loc('SuperTrend')] = curr_st
/tmp/ipykernel_5244/4251973704.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '968.125' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weekly.iloc[i, weekly.columns.get_loc('SuperTrend')] = curr_st
/tmp/ipy

In [7]:
latest_supertrend = daily_with_weekly_st.groupby("Ticker").tail(1)

In [8]:
print(latest_supertrend)

             Date      Ticker      Open      High       Low     Close  \
169    2025-09-22   20MICRONS    230.00    232.05    227.30    228.35   
287    2025-09-22  21STCENMGM     52.00     53.45     51.81     52.51   
464    2025-09-22      360ONE   1071.00   1087.00   1042.20   1050.40   
641    2025-09-22   3IINFOLTD     25.00     25.01     24.20     24.31   
818    2025-09-22     3MINDIA  29830.00  30135.00  29725.00  29975.00   
...           ...         ...       ...       ...       ...       ...   
365730 2025-09-22        ZOTA   1440.00   1457.60   1415.10   1422.50   
365907 2025-09-22       ZUARI    274.85    276.45    268.10    269.20   
366084 2025-09-22    ZUARIIND    290.55    292.00    270.15    286.55   
366261 2025-09-22   ZYDUSLIFE   1037.30   1046.15   1032.30   1035.70   
366438 2025-09-22   ZYDUSWELL    517.00    517.00    491.90    497.60   

          Volume    SuperTrend  Trend  
169     113130.0    221.788000   True  
287       3632.0     59.876000  False  
464

In [9]:
price_threshold = 0.03
volume_threshold = 2.0

latest_date = df["Date"].max()

In [56]:
results = []

for ticker, group in df.groupby("Ticker"):
    group = group.sort_values("Date").reset_index(drop=True)

    # Calculate daily % price change (Close vs Previous Close)
    group["Pct_Change"] = group["Close"].pct_change()

    # Calculate rolling average volume (last 20 days)
    group["Avg_Volume"] = group["Volume"].rolling(window=20, min_periods=5).mean()

    # Signal: price up > threshold & volume > threshold * avg_volume
    group["Signal"] = (group["Pct_Change"] > price_threshold) & (group["Volume"] > volume_threshold * group["Avg_Volume"])

    results.append(group)

result_df = pd.concat(results)

signals = result_df[result_df["Signal"]]
latest_signals = signals[signals["Date"] == latest_date]

In [59]:
latest_signals = latest_signals.merge(
    NSE_ISIN[["SYMBOL","ISIN NUMBER"]],
    left_on="Ticker",
    right_on="SYMBOL",
    how="inner"
)

In [61]:
latest_signals = pd.merge(
    latest_signals,
    allstocks,
    left_on="ISIN NUMBER",         # column name in latest_signals
    right_on="ISIN",     # column name in allstocks
    how="left"           # inner join (only matching tickers)
)

In [63]:
latest_signals = latest_signals.merge(
    latest_supertrend[["Ticker","SuperTrend","Trend"]],
    on="Ticker",         # column name in latest_signals
    how="left"           # inner join (only matching tickers)
)

In [88]:
filtered.shape

(2, 58)

In [87]:
filtered = latest_signals[
    (latest_signals["Market Capitalization"] > 1000) &
    (latest_signals["PEG TTM PE to Growth"] > 0) &
    (latest_signals["PEG TTM PE to Growth"] < 2) &
    (latest_signals["ROCE Annual 3Yr Avg %"] > 15) &
    (latest_signals['Long Term Debt To Equity Annual'] < 0.5) &
    (latest_signals['Promoter holding latest %'] > 60) &
    (latest_signals['Promoter holding pledge percentage % Qtr'] < 0.01) &
    (latest_signals['Net Profit Qtr Growth YoY %'] > 0) &
    (latest_signals['Operating Revenue growth TTM %'] > 15) &
    (latest_signals['Cash EPS 5Yr Growth %'] > 15) &
    (latest_signals['EPS TTM Growth %'] > 15)
]

In [90]:
filtered.to_csv("my_data.csv", index=False)

In [25]:
print(allstocks[allstocks['NSE Code']=='GUJTHEM'])

Empty DataFrame
Columns: [Stock Name, NSE Code, BSE Code, Stock Code, ISIN, Industry Name, sector_name, Current Price, Current Price.1, Market Capitalization, Trendlyne Durability Score, Trendlyne Valuation Score, Trendlyne Momentum Score, DVM_classification_text, Prev Day Trendlyne Durability Score, Prev Day Trendlyne Valuation Score, Prev Day Trendlyne Momentum Score, Prev Week Trendlyne Durability Score, Prev Week Trendlyne Valuation Score, Prev Week Trendlyne Momentum Score, Prev Month Trendlyne Durability Score, Prev Month Trendlyne Valuation Score, Prev Month Trendlyne Momentum Score, Normalized Momentum Score, Market Capitalization.1, PE TTM Price to Earnings, PEG TTM PE to Growth, Price to Book Value, Operating Revenue TTM, Revenue Growth Qtr YoY %, Operating Revenue growth TTM %, EPS TTM Growth %, Cash EPS 5Yr Growth %, Net Profit Qtr Growth YoY %, ROCE Annual 3Yr Avg %, Operating Profit Margin Qtr %, Operating Profit Margin TTM %, Operating Profit Margin Annual %, Operating P

In [19]:
allstocks.shape

(5693, 44)